In [ ]:
import os
import sys
import gdown
import shutil

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.discriminant_analysis import StandardScaler
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import set_config



In [ ]:
output = 'SBAnational.csv'

if not os.path.isfile(output):
    drive_url = 'https://drive.google.com/uc?id=1vdNaWcJgMzUnlaDVA4FuZRFYinO8UsPQ'
    gdown.download(drive_url, output, quiet=False)

shutil.copy(output, 'DuplicateSBAnational.csv') # pour copier le csv et travailler sur un duplicata

df = pd.read_csv('DuplicateSBAnational.csv')

In [ ]:
y = df['Approve']
X = df.drop('Approve', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("nums", numerical_preprocessor, numerical_columns_selector),
        ("cat", categorical_preprocessor, categorical_columns_selector)
    ]
)

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(max_depth=25, min_samples_leaf=80, min_samples_split=3, n_estimators=70, max_samples=0.7, n_jobs=-1 ,random_state=42))
])

In [ ]:
set_config(transform_output="pandas")

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.3f}')

In [ ]:
base_rf_classifier = RandomForestClassifier(max_depth=25, min_samples_leaf=80, min_samples_split=3, 
                                             n_estimators=70, max_samples=0.7, random_state=42)

bagging_classifier = BaggingClassifier(base_estimator=base_rf_classifier, n_jobs=-1, random_state=42)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', bagging_classifier)
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy:.3f}')